##1- Dataset Loading

In [ ]:
!pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
emotions = load_dataset("dair-ai/emotion")
emotions

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

## 2 - Understanding the Dataset

In [ ]:
emotions["train"][0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [ ]:
emotions["train"].column_names

['text', 'label']

In [ ]:
emotions["train"].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

## 3 - Data Preprocessing

In [ ]:
from transformers import AutoTokenizer
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

##3.1 - Padding

In [ ]:
from transformers import DataCollatorWithPadding

In [ ]:
data_collator=DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

##3.2 - Evaluate

In [ ]:
!pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00


In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions,
                            references = labels)

## 4 - Model Training

In [ ]:
from transformers import TFAutoModelForSequenceClassification

id2label={0:"sadness",1:"joy",2:"love",3:"anger",4:"fear",5:"suprise"}
label2id={"sadness":0,"joy":1,"love":2,"anger":3,"fear":4,"suprise":5}

model=TFAutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=6, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## 4.1 - Preparing the Datasets

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    emotions_encoded["train"],
    shuffle = True,
    batch_size = 16,
    collate_fn = data_collator
)

tf_validation_set = model.prepare_tf_dataset(
    emotions_encoded["validation"],
    shuffle = True,
    batch_size = 16,
    collate_fn = data_collator
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
import tensorflow as tf

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer)

In [ ]:
!pip install git-lfs

In [ ]:
!git lfs install

Git LFS initialized.


In [ ]:
!git config --global user.email "sevvalkapcak@gmail.com"
!git config --global user.name "sevvalkapcak"

In [ ]:
!git config --global credential.helper store

## 4.2 - Setting the Callbacks

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback=KerasMetricCallback(
    metric_fn=compute_metrics,
    eval_dataset = tf_validation_set
)
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="sevvalkapcak/newModel2",  # Namespace ve model adı
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/sevvalkapcak/newModel2 into local empty directory.


**NOT**

Eğitimimiz uzun süreceği için google collab bizi serverdan atabilir. Bunun önüne geçmek için aktif olduğumuzu bir şekile bildirmeliyiz.

Bunun için de sayfanın üst tarafına sağ tıklayıp "ögeyi denetle" veya "incele" seçeneğiniz seçip, çıkan pencereden "console"'a tıklayıp açılan komut satırına aşağıdaki kodu ekleyip enter tuşuna basarsak bu kod bizim 10 dakikada bir connect butonuna basarak bizim aktif kalmamızı sağlayacaktır.

function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)

In [ ]:
model.fit(x=tf_train_set,
         validation_data=tf_validation_set,
         epochs=100,
         callbacks=[metric_callback, push_to_hub_callback])

Epoch 1/100
1000/1000 [==============================] - 130s 130ms/step - loss: 0.2465 - val_loss: 0.2029 - accuracy: 0.9085
Epoch 2/100
1000/1000 [==============================] - 133s 133ms/step - loss: 0.1354 - val_loss: 0.1302 - accuracy: 0.9390
Epoch 3/100
1000/1000 [==============================] - 130s 129ms/step - loss: 0.1121 - val_loss: 0.1588 - accuracy: 0.9340
Epoch 4/100
1000/1000 [==============================] - 132s 132ms/step - loss: 0.0945 - val_loss: 0.1551 - accuracy: 0.9370
Epoch 5/100
1000/1000 [==============================] - 138s 138ms/step - loss: 0.0815 - val_loss: 0.1696 - accuracy: 0.9390
Epoch 6/100
1000/1000 [==============================] - 132s 132ms/step - loss: 0.0778 - val_loss: 0.1647 - accuracy: 0.9320
Epoch 7/100
1000/1000 [==============================] - 135s 135ms/step - loss: 0.0522 - val_loss: 0.2356 - accuracy: 0.9310
Epoch 8/100
1000/1000 [==============================] - 132s 132ms/step - loss: 0.0444 - val_loss: 0.2861 - accuracy:

## 5 - Prediction

In [ ]:
custom_text = "I watched a movie yesterday. It was really awesome"

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="sevvalkapcak/newModel2")

In [ ]:
inputs=tokenizer(custom_text, return_tensors= "tf")
logits = model(**inputs).logits
predicted_class_id=int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]

'joy'